<center>
    <h1>Traffic Flow Analysis of Cincinnati</h1>
    <h2>Network Analysis</h2>
    <h3>Mohammad Reza Ghasemi Madani and Riccardo Spolaor</h3>
    <h4> mohammadreza.ghasemi@studio.unibo.it, riccardo.spolaor@studio.unibo.it</h4>
</center>

---

The study's objective is to replicate and extend the Stanford study [*Traffic Flow Analysis Using
Uber Movement Data*](http://snap.stanford.edu/class/cs224w-2017/projects/cs224w-11-final.pdf) considering the city of Cincinnati.

The analysis is divided into two steps. Firstly, node centrality is computed in order to investigate traffic behavior throughout different times of the day in comparison to the topographic structure of the cities. Next, community detection is applied to identify travel clusters and regions with similar characteristics.

In [1]:
import numpy as np
import networkx as nx

In [2]:
# Settings for autoreloading

%load_ext autoreload
%autoreload 2

In [3]:
from random import seed

# Settings for riproducibility
SEED = 42
np.random.seed(SEED)
seed(SEED)

In [4]:
import os

# Important constants
CITY_NAME = 'Cincinnati'
GEOJSON_PATH = os.path.join('data', 'cincinnati_censustracts.json')
TRAVEL_TIMES_DATA_PATH = os.path.join('data', 'cincinnati-censustracts-2020-1-OnlyWeekdays-HourlyAggregate.csv')

# 01 Networks Definition
In this section the temporal network defining the mean travel times during weekdays at different hours is built along with a spatial graph defining the city structure.

## 01.1 Spatial Network
A spatial network is built as an undirected graph where the node correspond to different blocks of the cities and the edges contain the distance between neighboring regions.

A `GeoJSON` file describing the coordinates of the polygons which delimit the zones in the city is imported as a `GeoDataFrame` and  exploited to build the spatial network that models the underlying city structure.

In [5]:
from utils.geodataframe import get_geodataframe

gdf = get_geodataframe(GEOJSON_PATH)

print(f'Geodataframe of {CITY_NAME}:')
gdf.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data\\cincinnati_censustracts.json'

Centroids are computed for each area. They are subsequently used to calculate the distance among the areas and are used as reference points.

In [ ]:
from utils.geodataframe import set_geodataframe_centroids

set_geodataframe_centroids(gdf)

In [ ]:
print(f'Geodataframe of {CITY_NAME} after the centroid computation:')
gdf.head()

The spatial network is built from the `GeoDataFrame` and its statistics are illustrated.

In [ ]:
from utils.spatial_network import get_spatial_network

spatial_network = get_spatial_network(gdf)

In [ ]:
print(f'Number of nodes in the spatial network: {len(spatial_network.nodes())}')
print(f'Number of edges in the spatial network: {len(spatial_network.edges())}')
print(f'Spatial network density: {nx.density(spatial_network)}')

In [ ]:
print(f'The spatial network is directed: {nx.is_directed(spatial_network)}')

Finally, the spatial network is shown below over the city map.

In [ ]:
from utils.graphics import plot_spatial_network

plot_spatial_network(gdf, spatial_network, title=f"{CITY_NAME}'s spatial network representation")

# 01.2 Temporal Network
A temporal Network with dynamic weights on the edges is defined. The nodes correspond to different regions and are a subset of the ones
present in the spatial network, while the edges include the average travel time over the different considered hours of the day across the regions. The dynamicity of the temporal network is handled by building a dictionary that for each considered hour, contains the view of the temporal network at that specific hour.

A `pandas DataFrame` containing the mean travel times among regions is initially created from the `.csv` of the *Uber Movement* data of the city.

In [ ]:
from utils.temporal_network import get_movement_dataframe

df = get_movement_dataframe(TRAVEL_TIMES_DATA_PATH)

print(f"{CITY_NAME}'s movement dataframe:")
df.head()

The temporal network is built from the `DataFrame` and its statistics are illustrated.

In [ ]:
from utils.temporal_network import get_temporal_networks_from_pandas_edgelist

temporal_networks_dict = get_temporal_networks_from_pandas_edgelist(df, [0, 8, 13, 20])

In [ ]:
for k, v in temporal_networks_dict.items():
   print(f'Temporal network at time {k}: nodes: {len(v.nodes())} edges: {len(v.edges())} density: {nx.density(v)}') 

In [ ]:
for k, v in temporal_networks_dict.items():
    print(f'Temporal network at time {k} is directed: {nx.is_directed(v)}') 

# 02 Analysis
Centrality and community detection algorithms are applied on both the spatial and temporal networks of the city in order to extract insights about the traffic flow.

## 02.1 Node centrality
Centrality metrics are examined to identify bottlenecks or peak traffic zones in the temporal network and core/peripheral zones in the spatial network, aiming to better understand the structural characteristics of urban traffic flow and human activity in the considered cities.

### 02.1.1 Node degree
In temporal graphs nodes with high weighted *in-degree* and *out-degree* are likely to correspond to areas of high traffic congestion. Specifically, high *in-degree* nodes indicate areas that are the destination of many travelers, while high *out-degree*
nodes indicate areas that are the origin of many travels.

#### In-degree
Firstly, the *in-degree* of the nodes in temporal network is computed and the results are illustrated. 

In [ ]:
from utils.metrics import get_nodes_in_degree_centrality

temporal_networks_in_degrees = {
    k: get_nodes_in_degree_centrality(v, weight='mean_travel_time', normalize=False)
    for k, v in temporal_networks_dict.items()
}

In [ ]:
from utils.graphics import plot_normalized_temporal_centralities

plot_normalized_temporal_centralities(gdf, spatial_network, temporal_networks_in_degrees, CITY_NAME, 'in-degree')

In [ ]:
from utils.graphics import plot_temporal_network_centrality_distribution

plot_temporal_network_centrality_distribution(temporal_networks_in_degrees, 'in-degree centrality')

#### Out-degree
Next, the *out-degree* of the nodes in temporal network is computed and the results are illustrated. 

In [ ]:
from utils.metrics import get_nodes_out_degree_centrality

temporal_networks_out_degrees = {
    k: get_nodes_out_degree_centrality(v, weight='mean_travel_time', normalize=False)
    for k, v in temporal_networks_dict.items()
}

In [ ]:
from utils.graphics import plot_normalized_temporal_centralities

plot_normalized_temporal_centralities(gdf, spatial_network, temporal_networks_out_degrees, CITY_NAME, 'out-degree')

In [ ]:
from utils.graphics import plot_temporal_network_centrality_distribution

plot_temporal_network_centrality_distribution(temporal_networks_out_degrees, 'out-degree centrality')

### 02.1.2 Node betweenness
*Betweenness centrality* is a measure of a node's importance in a network based on the number of shortest paths that pass through it. Hence, nodes with high *betweenness centrality* are considered to be structurally important for connecting regions in the spatial graph. In the temporal graph the betweenness values of the nodes can be seen as traffic movement. Changes in this metric during different hours of the day can evaluate traffic movement and predict traffic flow.

#### Temporal Network
Firstly, the *betweenness* of the nodes is computed on the temporal network and the results are illustrated.

In [ ]:
from utils.metrics import get_nodes_betweenness_centrality

temporal_networks_betweenness = {
    k: get_nodes_betweenness_centrality(v, weight='mean_travel_time', normalize=False)
    for k, v in temporal_networks_dict.items()
}

In [ ]:
from utils.graphics import plot_normalized_temporal_centralities

plot_normalized_temporal_centralities(gdf, spatial_network, temporal_networks_betweenness, CITY_NAME, 'betweenness')

In [ ]:
from utils.graphics import plot_temporal_network_centrality_distribution

plot_temporal_network_centrality_distribution(temporal_networks_betweenness, 'betweenness centrality')

#### Spatial Network
Next, the *betweenness* of the nodes is computed on the spatial network and the results are illustrated.

In [ ]:
from utils.metrics import get_nodes_betweenness_centrality

spatial_network_betweenness = get_nodes_betweenness_centrality(spatial_network, weight='weight', normalize=True)

In [ ]:
from utils.graphics import plot_centrality

plot_centrality(gdf, spatial_network, spatial_network_betweenness,
                title = f"{CITY_NAME}'s betweenness centrality based on city distance")

In [ ]:
from utils.graphics import plot_spatial_network_centrality_distribution

plot_spatial_network_centrality_distribution(spatial_network_betweenness, 'betweenness centrality')

## 02.1.3 Node Closeness
*Closeness centrality* measures nodes' importance in a network based on how quickly it can reach all other nodes in the network. Regions with high *closeness centrality* can quickly access other areas in the network. So in the spatial and temporal graph the nodes in the center should have the highest closeness centrality scores.

In [ ]:
from utils.metrics import get_nodes_closeness_centrality

temporal_networks_closeness = {
    k: get_nodes_closeness_centrality(v, weight='mean_travel_time', normalize=False)
    for k, v in temporal_networks_dict.items()
}

In [ ]:
from utils.graphics import plot_normalized_temporal_centralities

plot_normalized_temporal_centralities(gdf, spatial_network, temporal_networks_closeness, CITY_NAME, 'closeness')


In [ ]:
from utils.graphics import plot_temporal_network_centrality_distribution

plot_temporal_network_centrality_distribution(temporal_networks_closeness, 'closeness centrality')

#### Spatial Network
Next, the *closeness* of the nodes is computed on the spatial network and the results are illustrated.

In [ ]:
from utils.metrics import get_nodes_closeness_centrality

spatial_network_closeness = get_nodes_closeness_centrality(spatial_network, weight='weight', normalize=True)

In [ ]:
from utils.graphics import plot_centrality

plot_centrality(gdf, spatial_network, spatial_network_closeness,
                title=f"{CITY_NAME}'s closeness centrality based on city distance")


In [ ]:
from utils.graphics import plot_spatial_network_centrality_distribution

plot_spatial_network_centrality_distribution(spatial_network_closeness, 'closeness centrality')

### 02.1.4 Nodes PageRank
*PageRank centrality* considers a node's importance based on the number and quality of links pointing to it. In the context of traffic analysis, links to central regions count more compared to the low transit regions. The *PageRank* of nodes in the temporal graph reflects the amount of traffic directed to them or adjacent regions and it can be employed to locate hotspots in cities during various times of the day. In the spatial graph, it highlights zones that connect densely joined regions.

#### Temporal Network
Firstly, the *PageRank* of the nodes is computed on the temporal network and the results are illustrated.

In [ ]:
from utils.metrics import get_nodes_pagerank_centrality

temporal_networks_pagerank = {
    k: get_nodes_pagerank_centrality(v, weight='mean_travel_time', normalize=False)
    for k, v in temporal_networks_dict.items()
}

In [ ]:
from utils.graphics import plot_normalized_temporal_centralities

plot_normalized_temporal_centralities(gdf, spatial_network, temporal_networks_pagerank, CITY_NAME, 'PageRank')

In [ ]:
from utils.graphics import plot_temporal_network_centrality_distribution

plot_temporal_network_centrality_distribution(temporal_networks_pagerank, 'PageRank centrality')

#### Spatial Network
Next, the *PageRank* of the nodes is computed on the spatial network and the results are illustrated.

In [ ]:
from utils.metrics import get_nodes_pagerank_centrality

spatial_network_pagerank = get_nodes_pagerank_centrality(spatial_network, weight='weight', normalize=True)

In [ ]:
from utils.graphics import plot_centrality

plot_centrality(gdf, spatial_network, spatial_network_pagerank,
                title=f"{CITY_NAME}'s PageRank centrality based on city distance")


In [ ]:
from utils.graphics import plot_spatial_network_centrality_distribution

plot_spatial_network_centrality_distribution(spatial_network_pagerank, 'PageRank centrality')

### 02.1.5 HITS measure for Nodes Hubs and Authorities
It gives each node in a directed network two centrality scores: the *Authority* and the *Hub* centrality. High authority nodes are the "more resourceful", hence the ones receiving more links. Differently, high hub scores are given to nodes that connect to high authority nodes. In the temporal graphs, authority can be interpreted as a measure of total time invested to reach a given region, while hub scores are a measure of the total time spent to drive to main travel regions. The results of this metric on the spatial graph are not
particularly important.

In [ ]:
from utils.metrics import get_nodes_hits_centrality

temporal_networks_hubs = dict()
temporal_networks_authorities = dict()

for k, v in temporal_networks_dict.items():
    hubs, authorities = get_nodes_hits_centrality(v, normalize=True, weight='mean_travel_time')
    temporal_networks_hubs[k] = hubs
    temporal_networks_authorities[k] = authorities

#### Temporal Network Hubs
Firstly, the *Hubs* of the nodes is computed on the temporal network and the results are illustrated.

In [ ]:
from utils.graphics import plot_normalized_temporal_centralities

plot_normalized_temporal_centralities(gdf, spatial_network, temporal_networks_hubs, CITY_NAME, 'Hubs')

In [ ]:
from utils.graphics import plot_temporal_network_centrality_distribution

plot_temporal_network_centrality_distribution(temporal_networks_hubs, 'hubs centrality')

#### Temporal Network Authorities
Next, the *Authorities* of the nodes is computed on the temporal network and the results are illustrated.

In [ ]:
from utils.graphics import plot_normalized_temporal_centralities

plot_normalized_temporal_centralities(gdf, spatial_network, temporal_networks_authorities, CITY_NAME, 'Authorities')

In [ ]:
from utils.graphics import plot_temporal_network_centrality_distribution

plot_temporal_network_centrality_distribution(temporal_networks_authorities, 'authorities centrality')

#### Spatial Network Hubs and Authorities
Finally, the *Hubs* and *Authorities* of the nodes are computed on the spatial network and the results are illustrated.

In [ ]:
from utils.metrics import get_nodes_hits_centrality

spatial_network_hits = get_nodes_hits_centrality(spatial_network, weight='weight', normalize=False)

In [ ]:
from utils.graphics import plot_centrality

plot_centrality(gdf, spatial_network, spatial_network_hits[0], 
                title = f"{CITY_NAME}'s Hubs centrality based on city distance")

plot_centrality(gdf, spatial_network, spatial_network_hits[1], 
                title = f"{CITY_NAME}'s Authorities centrality based on city distance")


In [ ]:
from utils.graphics import plot_spatial_network_centrality_distribution

plot_spatial_network_centrality_distribution(spatial_network_hits[0], 'hubs centrality')
plot_spatial_network_centrality_distribution(spatial_network_hits[1], 'authorities centrality')

## 02.2 Community Detection
The community detection algorithms aim to discover groups within the networks as part of our objective to reveal insights about the traffic flow structure. The results on the temporal graphs may help locate specific areas within the cities which display similar traffic behavior
during certain times of the day. Regarding the spatial network, the found communities should indicate structurally similar areas. As explained in the Dataset section we applied community detection on an undirected version of the temporal graphs.

In [ ]:
# Build a directory to save the results
RESULTS_DIR = 'results'
os.makedirs(RESULTS_DIR, exist_ok=True)

The community detection is applied on an undirected version of the temporal network for complexity and algorithmic compatibility resons. The undirected version of the temporal network is built below and its characteristics are illustrated

In [ ]:
undirected_temporal_networks_dict = {k: v.to_undirected() for k, v in temporal_networks_dict.items()}

In [ ]:
for k, v in undirected_temporal_networks_dict.items():
   print(f'Undirected temporal network at time {k}: nodes: {len(v.nodes())} edges: {len(v.edges())}') 

In [ ]:
for k, v in undirected_temporal_networks_dict.items():
    print(f'Undirected temporal network at time {k} density: {nx.density(v)}') 

### 02.2.1 Girvan-Newmann Strength ofWeak Ties algorithm
In this algorithm the edges with the highest weighted betweenness are gradually removed and the measure is recomputed after every removal until the network falls off to smaller components. In the temporal graph the obtained communities should consist in travel clusters which can
be connected through others by highly traffic-busy edges. In the spatial graph it should define densely connected areas which are highly distant from each other.

In [ ]:
from utils.metrics import get_girvan_newman_communities
from time import time

temporal_networks_girvan_newman_communities = dict()
times = dict()

for k, v in undirected_temporal_networks_dict.items():
    start_time = time()
    temporal_networks_girvan_newman_communities[k] = get_girvan_newman_communities(v, weight='mean_travel_time', k=4, seed=42)
    times[k] = time() - start_time

for k, t in times.items():
    print(f'Time for computing the temporal network communities at time {k:02d}:00:', f'{t:.2f} s')
print()
print(f'Total for computing the temporal network communities at time {k:02d}:00:', f'{sum(times.values()):.2f} s')

In [ ]:
from utils.graphics import plot_communities

for k, v in temporal_networks_girvan_newman_communities.items():
    plot_communities(gdf, spatial_network, v, title = f"{CITY_NAME}'s Girvan-Newman communities at {k:02d}:00")

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-girvan-newman-temporal.pickle'), 'wb') as f:
    pickle.dump(temporal_networks_girvan_newman_communities, f)

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-girvan-newman-temporal.pickle'), 'rb') as f:
    temporal_networks_girvan_newman_communities = pickle.load(f)

In [ ]:
from utils.metrics import get_girvan_newman_communities

start_time = time()
spatial_network_girvan_newman_communities = get_girvan_newman_communities(spatial_network, weight='weight', k=4, seed=42)
print(f'Time for computing the spatial network communities', f'{time() - start_time:.2f} s')

In [ ]:
from utils.graphics import plot_communities

plot_communities(gdf, spatial_network, spatial_network_girvan_newman_communities, 
                 title = f"{CITY_NAME}'s Girvan-Newman communities based on city distance")

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-girvan-newman-spatial.pickle'), 'wb') as f:
    pickle.dump(spatial_network_girvan_newman_communities, f)

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-girvan-newman-spatial.pickle'), 'rb') as f:
    spatial_network_girvan_newman_communities = pickle.load(f)

### 02.2.2 Community Detection based onWeighted K-Cores
A *weighted k-core* is a connected set of nodes where each has a weighted degree of value at least *k*. Once a *k-core*
community is computed, the algorithm is repeated for the remaining subgraph until all nodes are assigned to a community. This algorithm should define in the temporal graph similarly connected traffic-busy clusters, while it should create a partition of the spatial network based on groups of connected zones with similar distance between each other.

In [ ]:
from utils.metrics import get_k_cores_communities
from time import time

temporal_networks_k_core_communities = dict()
times = dict()

for k, v in undirected_temporal_networks_dict.items():
    start_time = time()
    temporal_networks_k_core_communities[k] = get_k_cores_communities(v, weight='mean_travel_time')
    times[k] = time() - start_time

for k, t in times.items():
    print(f'Time for computing the temporal network communities at time {k:02d}:00:', f'{t:.2f} s')
print()
print(f'Total for computing the temporal network communities at time {k:02d}:00:', f'{sum(times.values()):.2f} s')

In [ ]:
from utils.graphics import plot_communities

for k, v in temporal_networks_k_core_communities.items():
    plot_communities(gdf, spatial_network, v, title = f"{CITY_NAME}'s k-core communities at {k:02d}:00")

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-k-core-temporal.pickle'), 'wb') as f:
    pickle.dump(temporal_networks_k_core_communities, f)

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-k-core-temporal.pickle'), 'rb') as f:
    temporal_networks_k_core_communities = pickle.load(f)

In [ ]:
from utils.metrics import get_k_cores_communities

start_time = time()
spatial_network_k_core_communities = get_k_cores_communities(spatial_network, weight='weight')
print(f'Time for computing the spatial network communities', f'{time() - start_time:.2f} s')

In [ ]:
from utils.graphics import plot_communities

plot_communities(gdf, spatial_network, spatial_network_k_core_communities, 
                 title = f"{CITY_NAME}'s k-core communities based on city distance")

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-k-core-spatial.pickle'), 'wb') as f:
    pickle.dump(spatial_network_k_core_communities, f)

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-k-core-spatial.pickle'), 'rb') as f:
    spatial_network_k_core_communities = pickle.load(f)

### 02.2.3 Clique Percolation Method
A community detection algorithm that computes communities from *k-cliques*. In addition, in order for a k-clique to be considered, the *geometric mean* among the weight of its edges (or *intensity*) should be less or equal than a threshold *l*. The algorithm is repeated in the remaining subgraph until all nodes are assigned to a community. It should identify group of nodes more densely connected between each
others, such as connected regions in the spatial graph or zones connected by travels in the temporal ones. The intensity component should penalize weak connections in the communities.

In [ ]:
from utils.metrics import get_clique_percolation_communities
from time import time

temporal_networks_clique_percolation_communities = dict()
times = dict()

for k, v in undirected_temporal_networks_dict.items():
    start_time = time()
    temporal_networks_clique_percolation_communities[k] = get_clique_percolation_communities(v, weight='mean_travel_time', k=4)
    times[k] = time() - start_time

for k, t in times.items():
    print(f'Time for computing the temporal network communities at time {k:02d}:00:', f'{t:.2f} s')
print()
print(f'Total for computing the temporal network communities at time {k:02d}:00:', f'{sum(times.values()):.2f} s')

In [ ]:
from utils.graphics import plot_communities

for k, v in temporal_networks_clique_percolation_communities.items():
    plot_communities(gdf, spatial_network, v,
                     title=f"{CITY_NAME}'s Clique Percolation communities at {k:02d}:00")

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-cliques-percolation-temporal.pickle'), 'wb') as f:
    pickle.dump(temporal_networks_clique_percolation_communities, f)

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-cliques-percolation-temporal.pickle'), 'rb') as f:
    temporal_networks_clique_percolation_communities = pickle.load(f)

In [ ]:
from utils.metrics import get_clique_percolation_communities

start_time = time()
spatial_network_clique_percolation_communities = get_clique_percolation_communities(spatial_network, weight='weight', k=2)
print(f'Time for computing the spatial network communities', f'{time() - start_time:.2f} s')

In [ ]:
from utils.graphics import plot_communities

plot_communities(gdf, spatial_network, spatial_network_clique_percolation_communities, 
                 title = f"{CITY_NAME}'s Clique Percolation communities based on city distance")

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-cliques-percolation-spatial.pickle'), 'wb') as f:
    pickle.dump(spatial_network_clique_percolation_communities, f)

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-cliques-percolation-spatial.pickle'), 'rb') as f:
    spatial_network_clique_percolation_communities = pickle.load(f)

### 02.2.4 Louvain Community Detection Algorithm
A heuristic community detection algorithm that computes communities by optimizing the *modularity* of the network. Given this premise, the obtained partition will obviously be the ones reching the highest modularity scores. Hence, the results of this methodology are used as a baseline to compare the quality of the other community detection algorithms.

In [ ]:
from utils.metrics import get_louvain_communities
from time import time

temporal_networks_louvain_communities = dict()
times = dict()

for k, v in undirected_temporal_networks_dict.items():
    start_time = time()
    temporal_networks_louvain_communities[k] = get_louvain_communities(v, weight='mean_travel_time')
    times[k] = time() - start_time

for k, t in times.items():
    print(f'Time for computing the temporal network communities at time {k:02d}:00:', f'{t:.2f} s')
print()
print(f'Total for computing the temporal network communities at time {k:02d}:00:', f'{sum(times.values()):.2f} s')

In [ ]:
from utils.graphics import plot_communities

for k, v in temporal_networks_louvain_communities.items():
    plot_communities(gdf, spatial_network, v,
                     title=f"{CITY_NAME}'s Louvain communities at {k:02d}:00")

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-louvain-temporal.pickle'), 'wb') as f:
    pickle.dump(temporal_networks_louvain_communities, f)

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-louvain-temporal.pickle'), 'rb') as f:
    temporal_networks_louvain_communities = pickle.load(f)

In [ ]:
from utils.metrics import get_louvain_communities

start_time = time()
spatial_network_louvain_communities = get_louvain_communities(spatial_network, weight='weight')
print(f'Time for computing the spatial network communities', f'{time() - start_time:.2f} s')

In [ ]:
from utils.graphics import plot_communities

plot_communities(gdf, spatial_network, spatial_network_louvain_communities, 
                 title = f"{CITY_NAME}'s Louvain communities based on city distance")

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-louvain-spatial.pickle'), 'wb') as f:
    pickle.dump(spatial_network_louvain_communities, f)

In [ ]:
import pickle

with open(os.path.join(RESULTS_DIR, f'{CITY_NAME}-louvain-spatial.pickle'), 'rb') as f:
    spatial_network_louvain_communities = pickle.load(f)

### 02.2.5 Modularity comparison
The modularity measure has been considered as an objective evaluation metric for the quality of the communities. Networks with high modularity have dense connections between the nodes within modules but sparse connections between nodes in different modules.

The results can be observed below.

In [ ]:
from utils.graphics import plot_temporal_network_modularities

plot_temporal_network_modularities(undirected_temporal_networks_dict, CITY_NAME, temporal_networks_girvan_newman_communities,
                                   temporal_networks_k_core_communities, temporal_networks_clique_percolation_communities,
                                   temporal_networks_louvain_communities)

In [ ]:
from utils.graphics import plot_spatial_network_modularities

plot_spatial_network_modularities(spatial_network, CITY_NAME, spatial_network_girvan_newman_communities,
                                  spatial_network_k_core_communities, spatial_network_clique_percolation_communities,
                                  spatial_network_louvain_communities) 